In [1]:
#### Analysis focusing on the non-EU countries
#### and non-big players – that is, no Russia, Ukraine, China, US

In [2]:
import pandas as pd
import pycountry

In [3]:
# Data
df = pd.read_feather("../../output/for-analysis.feather")

In [4]:
# EU members plus USA, China, Russia, Ukraine
big_players = [ "RUS", "UKR", "USA", "CHN" ]

eu_members = [
    "AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", "FRA", 
    "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", "MLT", "NLD", 
    "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE",    
]

In [5]:
# All others (non-eu + non-big players)
all_others = [ item for item in df.columns.tolist()[17:] if item not in big_players and item not in eu_members ]

# All non-eu members
non_eu = [ item for item in df.columns.tolist()[17:] if item not in eu_members ]

In [6]:
df[big_players + eu_members + non_eu].sum().drop_duplicates().sort_values(ascending=False).head(50)

UKR    5510
RUS    4366
CHN    2574
POL    2420
FRA    1896
HUN    1831
DEU    1662
ESP    1546
USA    1533
ITA    1515
TUR    1510
GRC    1136
ROU    1045
IRL     948
IRN     850
BGR     784
BLR     754
SYR     695
GBR     669
SWE     622
PRT     588
MDA     563
AFG     550
NLD     489
MLT     487
HRV     458
BEL     454
IND     450
LBY     430
THA     407
GEO     401
SVK     392
AZE     377
AUT     360
VEN     354
PSE     344
CYP     324
SRB     321
LTU     307
MKD     298
IRQ     293
FIN     291
MLI     287
BRA     284
ARM     283
ALB     279
DNK     278
QAT     275
TUN     257
SVN     256
dtype: int64

In [7]:
# Countries with the most mentions among all others
df[all_others].sum().sort_values(ascending=False).head(50)

TUR    1510
ISR    1136
IRN     850
BLR     754
SYR     695
GBR     669
MDA     563
AFG     550
IND     450
LBY     430
THA     407
GEO     401
AZE     377
VEN     354
PSE     344
SRB     321
MKD     298
IRQ     293
MLI     287
BRA     284
ARM     283
ALB     279
QAT     275
TUN     257
EGY     239
MAR     235
BIH     217
AUS     212
CUB     200
DMA     196
CAN     191
ISL     184
LBN     183
CHL     177
NIC     173
YEM     165
JPN     159
PAK     154
JOR     153
NOR     151
NGA     142
SDN     140
SAU     140
MEX     139
NZL     137
VNM     133
ETH     132
CHE     129
MMR     113
COL     112
dtype: int64

In [53]:
df.shape[0]

58429

In [54]:
# Same, but by share of total speeches
(df[all_others].sum().sort_values(ascending=False) / df.shape[0] * 100).head(30)

TUR    2.584333
ISR    1.944240
IRN    1.454757
BLR    1.290455
SYR    1.189478
GBR    1.144979
MDA    0.963563
AFG    0.941313
IND    0.770166
LBY    0.735936
THA    0.696572
GEO    0.686303
AZE    0.645228
VEN    0.605864
PSE    0.588749
SRB    0.549385
MKD    0.510021
IRQ    0.501463
MLI    0.491194
BRA    0.486060
ARM    0.484349
ALB    0.477503
QAT    0.470657
TUN    0.439850
EGY    0.409043
MAR    0.402198
BIH    0.371391
AUS    0.362834
CUB    0.342296
DMA    0.335450
dtype: float64

In [52]:
df.shape[0] * 0.77 / 100

449.9033

In [9]:
# Eu: how much of total?
df[eu_members].sum().sum() / df.shape[0] * 100

35.612452720395694

In [10]:
# Non eu: how much of total?
df[non_eu].sum().sum() / df.shape[0] * 100

57.03332249396704

In [11]:
# Big-players: how much of total?
df[big_players].sum().sum() / df.shape[0] * 100

23.93160930359924

In [12]:
# Russia and Ukraine, how much of total?
df[['RUS', 'UKR']].sum().sum() / df.shape[0] * 100

16.902565506854472

In [13]:
# Usa and China, how much of total?
df[['USA', 'CHN']].sum().sum() / df.shape[0] * 100

7.029043796744768

In [14]:
# China, how much of total?
df[['CHN']].sum().sum() / df.shape[0] * 100

4.405346660048949

In [15]:
# I knew that Thailand was full of false positives before. Is this still the case?
for index, row in df[df.THA].iterrows():
    print(row.language)
    print(row.valid_tags)
    print(row.speech)

finnish
equality, legislation
Silvia Modig,  The Left -ryhmän puolesta. – Arvoisa puhemies, jokainen lapsi on yhtä arvokas ja jokaisen lapsen oikeuksien on toteuduttava yhdenvertaisesti riippumatta siitä, miten hän on saanut alkunsa, miten hän on syntynyt tai millaisessa perheessä hän elää. Tämä kuuluu lapsen oikeuksiin.  Nykytilanteessa sateenkaariperheet ovat eriarvoisessa asemassa muihin perheisiin nähden, koska he eivät voi olla varmoja siitä, että heidän vanhemmuutensa tunnustetaan uudessa kotimaassa heidän lähtömaansa tavoin. Vapaa liikkuvuus on kirjattu EU:n perussopimuksiin, mutta sateenkaariperheet eivät nykyoloissa voi muuttaa vapaasti jäsenmaasta toiseen ilman pelkoa juridisen vanhemmuuden menettämisestä.  Noin kaksi miljoonaa lasta EU:ssa on vaarassa menettää juridisen suhteen vanhempaansa, koska perheisiin sovellettavat lait ovat erilaisia eri jäsenmaissa. Neuvosto on nyt se, jonka käsissä on turvata näiden perheiden oikeudet. Neuvoston onkin kiireesti hyväksyttävä komissi

In [16]:
# It's a common language in Baltic languages.
# If we exclude them from the count, it's not in the top 10
# We will not mention it on the story.
df[df.THA].language.value_counts()

Lithuanian    193
finnish       135
lithuanian     45
english        13
latvian         9
german          5
french          2
italian         2
romanian        1
croatian        1
spanish         1
Name: language, dtype: int64

In [17]:
# India has a similar issue with the word 'indien', a common dutch term.
# However, it still stays high on top if we ignore the language.
df[df.IND].language.value_counts()

english         123
french           88
german           52
dutch            36
italian          35
spanish          29
portuguese       16
danish           14
polish           13
slovak           10
greek             9
croatian          6
czech             5
romanian          4
swedish           3
estonian          2
slovenian         1
multilingual      1
hungarian         1
bulgarian         1
finnish           1
Name: language, dtype: int64

In [18]:
# Manual inspection of samples of all other highlights gave us no clear indication of any other event like this.

In [19]:
# Total share of speeches that touch on each topic
df[df.columns[8:17]].mean()

Parliament issues and procedures            0.117750
Democracy and Governance                    0.463263
War and Security                            0.250646
International Relations and Diplomacy       0.088569
Economy, Finance and Infrastructure         0.400897
Environment and Climate                     0.221756
Health                                      0.083623
Technology and Innovation                   0.162608
Human Rights, Freedoms and Social Issues    0.357716
dtype: float64

In [20]:
### Manually inspecting a share of each country-topic pair of interest

In [21]:
#### UK
for index, row in df[(df.GBR) & (df['International Relations and Diplomacy'])].head(10).sample(frac=1).iterrows():
    print(row.language)
    print(row.valid_tags)
    print(row.speech)

portuguese
EU Parliament, Brexit, economy
Pedro Silva Pereira (S&D).   – Senhora Presidente da Comissão, dois pontos, o primeiro sobre o Reino Unido e a mensagem é simples, nós desejamos um bom acordo para uma relação futura que sirva os interesses das duas partes, mas a União Europeia não pode fazer o segundo acordo com quem diz que não vai cumprir o primeiro. É tão simples como isso.  Para que possa haver um novo acordo com o Reino Unido, como desejamos, é preciso que o Governo de Boris Johnson assegure o total respeito pelo acordo de saída que assinou e que é uma obrigação clara de direito internacional.  A segunda mensagem diz respeito ao seguinte: a União Europeia enfrenta muitos desafios, mas nenhum é mais decisivo do que estar à altura dos anseios dos cidadãos na resposta a esta crise económica devastadora causada pela pandemia. A proposta do Conselho pode e deve ser melhorada nas negociações com o Parlamento, mas todos devem recordar o essencial: é preciso que o dinheiro chegue

In [22]:
# India and Economy
for index, row in df[(df.IND) & (df['Economy, Finance and Infrastructure'])].sample(frac=1).head(10).iterrows():
    print(row.language)
    print(row.valid_tags)
    print(row.speech)
    print()

german
environment, economy
Malte Gallée (Verts/ALE).   – Herr Präsident, Herr Kommissar! Wenn man sich die Liste der Länder anschaut, die Tiefseebergbau unterstützen, stößt man auf China, Russland, Indien und Norwegen. Ich würde sagen, Norwegen ist da in ein bisschen kritischer Gesellschaft, und generell das Thema Tiefseebergbau sollten wir ganz, ganz, ganz schnell wieder von unserer Tagesordnung streichen.  Wir schaffen es ja jetzt noch nicht einmal, an Land wirklich konsequent hohe Umweltstandards zu etablieren. Wie soll das denn im Ozean funktionieren? Zudem brauchen wir das nicht. Genau solche Aktionen, die laufen doch unserer ganzen Idee einer ehrlichen Kreislaufwirtschaft zuwider, in der wir eben irgendwann unabhängig von diesen Quellen werden.  Wir stehen vor der riesengroßen Aufgabe, dass wir es endlich schaffen, die externalisierten Kosten unseres Wirtschaftssystems zu internalisieren. Und ich bin mir sicher, das ist verdammt schwierig in einer Region, wo wir noch nicht einma

In [23]:
df[(df.IND) & (df['Environment and Climate'])]

,level_0,speech,date,term,year,speech_length_in_characters,language,valid_tags,Parliament issues and procedures,Democracy and Governance,...,ISL,TUN,BLZ,CHN,MDG,KGZ,LCA,PAN,BHS,GTM
1530,11441,"Cindy Franssen, namens de PPE-Fractie. – Voor...",2023-10-02,9,2023,1869.0,dutch,"health, environment, legislation",True,False,...,False,False,False,False,False,False,False,False,False,False
4089,20408,"Mick Wallace (GUE/NGL). – Mr President, the ...",2019-11-28,9,2019,1052.0,english,environment,False,False,...,False,False,False,False,False,False,False,False,False,False
5362,31662,"Virginijus Sinkevičius, Member of the Commissi...",2020-10-21,9,2020,5786.0,english,"climate, biodiversity, legislation",True,False,...,False,False,False,True,False,False,False,False,False,False
5493,32653,"Stéphane Bijoux, au nom du groupe Renew. – Mo...",2022-06-06,9,2022,1233.0,french,"climate, energy, youth, employment, solidarity",False,False,...,False,False,False,False,False,False,False,False,False,False
7189,42404,"Sara Skyttedal (PPE). – Madam President, whe...",2022-06-07,9,2022,1380.0,english,"climate, industry, innovation",False,False,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57013,325634,"Sirpa Pietikäinen (PPE), kirjallinen. – On el...",2021-11-24,9,2021,626.0,finnish,"climate, finance, technology",False,False,...,False,False,False,True,False,False,False,False,False,False
57089,325725,"Christian Ehler, on behalf of the PPE Group. ...",2021-11-24,9,2021,2233.0,english,"climate, energy, innovation, industry, democracy",False,True,...,False,False,False,True,False,False,False,False,False,False
57126,325762,Robert Hajšel (S&D). – Vážená pán predsedajú...,2021-11-24,9,2021,1196.0,slovak,"climate, energy, renewable energy, social justice",False,False,...,False,False,False,False,False,False,False,False,False,False
58120,331242,Charles Goerens (Renew). – Madame la Préside...,2019-11-26,9,2019,2061.0,french,"security, research, climate",False,False,...,False,False,False,True,False,False,False,False,False,False


In [24]:
# India and Economy
for index, row in df[(df.IND) & (df['Environment and Climate'])].sample(frac=1).head(10).iterrows():
    print(row.language)
    print(row.valid_tags)
    print(row.speech)
    print()

slovak
climate, energy, renewable energy, social justice
Robert Hajšel (S&D).   – Vážená pán predsedajúci, najlacnejšia energia je stále tá, ktorú ušetríme a nemusíme vôbec vyrábať, a preto našou prioritou musí byť zvyšovanie energetickej efektívnosti. Buďme ale férový a priznajme si, že ak Únia dováža suroviny z tretích krajín, pri ktorých produkcii sa nerešpektujú naše zelené, ale ani sociálne normy, emisie iba vyvážame a podieľa sa na ich zvyšovaní inde. Ale planéta, na ktorej žijeme, je len jedna. U nás doma musíme rešpektovať rôzne štartovacie pozície jednotlivých štátov. V niektorých ešte pár rokov bude treba na rýchlejšiu dekarbonizáciu využívať aj plyn aj jadro. Samozrejme, prioritu musia dostať obnoviteľné zdroje, pričom do budúcnosti treba posilniť cezhraničné kapacity prenosu elektriky. Potreba posilniť odolnosť a energetickú bezpečnosť je o to naliehavejšia, že rastúce ceny energií ohrozujú zraniteľné domácnosti. Aj keď ľudia chápu potrebu znižovania emisií a zelených refor

#### Exporting data for charts

In [25]:
# Export for bar chart:
# Non-EU countries
def make_bar_chart(df, non_eu, eu_members):
    
    def get_country(iso3):
        if iso3 == 'IRN':
            country_name = 'Iran'

        elif iso3 == 'RUS':
            country_name = 'Russia'

        elif iso3 == 'SYR':
            country_name = 'Syria'

        elif iso3 == 'COD':
            country_name = 'DR Congo'

        elif iso3 == 'TUR':
            country_name = 'Turkey'
            
        elif iso3 == 'MDA':
            country_name = 'Moldova'
        
        else:
            country_name = pycountry.countries.get(alpha_3=iso3).name
            
        return country_name
    
    
    def get_iso2(iso3):
        return pycountry.countries.get(alpha_3=iso3).alpha_2.lower()

    data = df[non_eu].sum().sort_values(ascending=False).head(14).reset_index().rename(columns={'index':'iso3',
                                                                                                 0: 'count'})
    
    data['percentage'] = data['count'] / df.shape[0] * 100
    data['country'] = data.iso3.apply(get_country)
    data['iso2'] = data.iso3.apply(get_iso2)
    data['display_name'] = ":" + data.iso2 + ":" + " " + data.country
    
    return data

In [26]:
make_bar_chart(df, non_eu, eu_members).to_csv("../../viz-files/non-eu-ranking.csv", index=False)

#### Countries + big topics

In [27]:
# of_interest = df[all_others].sum().sort_values(ascending=False).head(10).index
# topics = df.columns[9:17].tolist()

# rows = []
# for country in of_interest:
    
        
#     subset = df[df[country]]
#     print(country)
    
#     if country == 'IRN':
#         #print('yoyo')
#         country_name = 'Iran'
        
#     elif country == 'RUS':
#         #print('yoyo')
#         country_name = 'Russia'
        
#     elif country == 'SYR':
#         #print('yoyo')
#         country_name = 'Syria'
        
#     elif country == 'COD':
#         #print('yoyo')
#         country_name = 'DR Congo'
        
#     elif country == 'TUR':
#         #print('yoyo')
#         country_name = 'Turkey'
        
#     elif country == 'MDA':
#         country_name = 'Moldova'
        
        
#     else:
#         country_name = pycountry.countries.get(alpha_3=country).name
        
#     print(country_name)
    
#     for topic in topics:
#         row = {}
        
#         # % about the issue
#         share = subset[topic].sum() / subset.shape[0] * 100
#         row['iso3'] = country
#         row['iso2'] = pycountry.countries.get(alpha_3=country).alpha_2
#         row['country'] = f":{row['iso2'].lower()}: {country_name}"
#         row['value_country'] = share
# #         row['% of all speeches that menion the topic'] = df[topic].sum() / df.shape[0] * 100
#         #row['value_total'] = df[topic].sum() / df.shape[0] * 100
#         row['topic'] = topic.lower()
#         row['label'] = f"Of all speeches that mention ___, how many are about {topic.lower()}?"
#         row['count_speeches'] = subset.shape[0]
        
#         rows.append(row)
        
# # Adds the overall value for all speeches
# for topic in topics:
#     row = {}
#     row['iso3'] = 'ALL'
#     row['country'] = 'All speeches'
#     row['topic'] = topic.lower()
#     row['label'] = f"Of all speeches that mention ___, how many are about {topic.lower()}?"
#     row['value_country'] = df[topic].sum() / df.shape[0] * 100
#     row['count_speeches'] = df.shape[0]
#     rows.append(row)

In [28]:
# topics_countries = pd.DataFrame(rows)

In [29]:
# for topic ain topics:
#     subset = topics_countries[topics_countries['topic']==topic.lower()]
#     subset.to_csv(f"../../viz-files/{ topic }-of-interest-bars.csv", index=False)

#### By continent, with EU and non-EU split

In [30]:
# Simple resampler for plotting data over time, using a 2 month window
def resampler(df, window='2M', agg='mean'):

    # Ensure the 'date' column is in datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Set the 'date' column as the index
    df = df.set_index('date')

    # Resample the dataframe to three-month intervals and calculate the share of True values (the mean works
    # because a True is counted as 1 and a False as 0). Summing all the 1s and dividing by total observations
    # equals to a percentage of total.
    if agg == 'mean':
        df = df.resample(window).agg(agg) * 100
    else:
        df = df.resample(window).agg(agg)
        
    # Custom date format function
    def custom_date_format(index):
        
        formatted_dates = []
        
        for date in index:
            
            start_date = date - pd.DateOffset(months=1)
            end_date = date
            formatted_dates.append(f"{start_date.strftime('%B %Y')} to {end_date.strftime('%B %Y')}")
            
        return formatted_dates
        
    # Adds a human readable tag for the datawrapper legend
    df['span'] = custom_date_format(df.index)
    
#     df.iloc[0, 'span'] = 'July 2019'
#     df.iloc[-1, 'span'] = 'April 2024'
    
    return df

In [31]:
# Continental overview – ISO3 to continent
correspondence = pd.read_csv("../../input/country-and-continent-codes-list.csv", sep=';')

In [32]:
# Keep only the iso codes that exist in the columns
correspondence = correspondence[correspondence['ISO-alpha3 code'].isin(df.columns)]

In [33]:
# A dict with the matches
continent_iso_map = correspondence.groupby('continent')['ISO-alpha3 code'].apply(list).to_dict()

In [34]:
continent_iso_map['European Union'] = eu_members

In [35]:
continent_iso_map['Europe, non-EU'] = [ country for country in continent_iso_map['Europe'] if country not in eu_members]

In [36]:
# Adds a column to check whether any country in the continent was mentioned
for continent, countries in continent_iso_map.items():
    df[continent] = df[['date'] + countries].sum(axis=1) > 0

/var/folders/yl/bd_xy2c53495c2z2g0w0yrzdtddnbq/T/ipykernel_29125/4118403187.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df[continent] = df[['date'] + countries].sum(axis=1) > 0
/var/folders/yl/bd_xy2c53495c2z2g0w0yrzdtddnbq/T/ipykernel_29125/4118403187.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df[continent] = df[['date'] + countries].sum(axis=1) > 0
/var/folders/yl/bd_xy2c53495c2z2g0w0yrzdtddnbq/T/ipykernel_29125/4118403187.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduc

In [37]:
resampler(df[['date', 'Africa', 'Americas', 'Asia', 'European Union', 'Europe, non-EU', 'Oceania']])

/var/folders/yl/bd_xy2c53495c2z2g0w0yrzdtddnbq/T/ipykernel_29125/421063775.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])


,Africa,Americas,Asia,European Union,"Europe, non-EU",Oceania,span
date,,,,,,,
2019-07-31,3.819444,11.284722,10.243056,27.951389,12.500000,0.347222,June 2019 to July 2019
2019-09-30,1.799775,9.111361,18.335208,20.359955,9.448819,0.000000,August 2019 to September 2019
2019-11-30,4.665461,7.450271,18.119349,24.050633,11.464738,0.397830,October 2019 to November 2019
2020-01-31,6.564715,7.758299,13.875420,24.207385,7.236106,2.275270,December 2019 to January 2020
2020-03-31,3.356555,4.116529,16.656111,24.192527,6.079797,0.126662,February 2020 to March 2020
2020-05-31,2.115385,2.692308,5.192308,24.615385,2.500000,0.000000,April 2020 to May 2020
2020-07-31,3.812636,6.971678,14.270153,22.657952,8.061002,0.435730,June 2020 to July 2020
2020-09-30,6.631763,2.966841,25.130890,26.701571,13.612565,0.349040,August 2020 to September 2020
2020-11-30,5.248991,6.415433,16.016151,23.642889,8.748318,0.269179,October 2020 to November 2020


In [38]:
resampler(df[['date', 'Africa', 'Americas', 'Asia', 'European Union', 'Europe, non-EU', 'Oceania']])\
    .reset_index()\
    .to_csv("../../viz-files/continents_2m_mean_eu_split.csv", index=False)

/var/folders/yl/bd_xy2c53495c2z2g0w0yrzdtddnbq/T/ipykernel_29125/421063775.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])


#### Topics over time

In [39]:
topics = df.columns[8:17].tolist()
resampler(df[ ['date'] + topics]).to_csv("../../viz-files/all-topics-multiple-bars.csv", index=False)

/var/folders/yl/bd_xy2c53495c2z2g0w0yrzdtddnbq/T/ipykernel_29125/421063775.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])


#### Most mentioned topics

In [47]:
(df[topics].sum() / df.shape[0]).to_frame().reset_index().rename(columns={'index':'Topic', 0:'share'})\
    .to_csv("../../viz-files/bar-chart-topics.csv", index=False)

#### Selected countries + sortable table

In [ ]:
groups = ['Parliament issues and procedures',
 'Democracy and Governance',
 'War and Security',
 'International Relations and Diplomacy',
 'Economy, Finance and Infrastructure',
 'Environment and Climate',
 'Health',
 'Technology and Innovation',
 'Human Rights, Freedoms and Social Issues']

In [ ]:
top_10 = df[all_others].sum().sort_values(ascending=False).head(10).index
rows = []
for country in top_10:
    
    row = {}
    
    subset = df[df[country]]
    
    if country == 'IRN':
        country_name = 'Iran'

    elif country == 'RUS':
        country_name = 'Russia'

    elif country == 'SYR':
        country_name = 'Syria'

    elif country == 'COD':
        country_name = 'DR Congo'

    elif country == 'TUR':
        country_name = 'Turkey'

    elif country == 'MDA':
        country_name = 'Moldova'
        
    else:
        country_name = pycountry.countries.get(alpha_3=country).name
    
    iso2 = pycountry.countries.get(alpha_3=country).alpha_2.lower()
    row['country_name'] = country_name
    row['display_name'] = f':{iso2}: {country_name}'
    row['category'] = 'country'
    
    for group in groups:
        
        # % about the issue
        share = subset[group].sum() / subset.shape[0] * 100
        row[group] = round(share, 2)
        
    rows.append(row)
        
        
    
# All speeches
row = {}
row['country_name'] = "All speeches"
row['display_name'] = "All speeches"
row['category'] = 'all'


for group in groups:

    # % about the issue
    share = df[group].sum() / df.shape[0] * 100
    row[group] = round(share, 2)


rows.append(row)

In [ ]:
pd.DataFrame(rows).to_csv("../../viz-files/table.csv", index=False)

In [ ]:
pd.DataFrame(rows)

#### Choropleth map with all countries

In [ ]:
#df.sum().mean()

In [ ]:
choropleth_data = (df[df.columns[17:212]].sum() / df.shape[0] * 100).reset_index()

In [ ]:
choropleth_data = choropleth_data.rename(columns={'index': 'iso3', 0: 'share'})

In [ ]:
def get_country(iso3):
    if iso3 == 'IRN':
        country_name = 'Iran'

    elif iso3 == 'RUS':
        country_name = 'Russia'

    elif iso3 == 'SYR':
        country_name = 'Syria'

    elif iso3 == 'COD':
        country_name = 'DR Congo'

    elif iso3 == 'TUR':
        country_name = 'Turkey'

    elif iso3 == 'MDA':
        country_name = 'Moldova'

    else:
        country_name = pycountry.countries.get(alpha_3=iso3).name

    return country_name

In [ ]:
choropleth_data['country_name'] = choropleth_data.iso3.apply(get_country)

In [ ]:
choropleth_data.to_csv("../../viz-files/choropleth.csv", index=False)